## Mirror-Image, Reverse-Decay Model for Multi-Touch Marketing Attribution

- Multi-touch marketing attribution is a technique used to weight specific marketing campaigns associated with customer sessions that lead to a purchase.
    - By multi-touch, we mean that we examine a series of User Sessions (or "touch points") associated that lead to a purchase.
    - Each Session is associated with a marketing campaign - e.g. A user clicks on an email to land on our ecommerce site.
    - Typically we think of the first session in time to be critical, as well as the session that occurs right before a purchase.
    - Sessions towards the middle of the series of sessions are viewed as having had less impact on the user's decision to make a purchase.
    - Rather than giving credit to a single marketing campaign for a given purchase, multi-touch models take into account several sessions leading up to a purchase.
- For this project, I demonstrate how to weight a 7-day history of user sessions leading to a purchase so that the first and last sessions are given the same weight, decaying exponentially to the middle session(s), which are weighted the least.
- To simplify the example, I will assign these generic digital marketing channels to each User Session without reference to specific campaigns.
    - (`'OrganicSearch', 'Direct', 'Social', 'PaidSearch', 'Email'`)
- In practice, marketing channels are simply a given medium (Email) used for a single marketing campaign (e.g., a Christmas 2023 Email Blast).

#### Let's Start with 2 Separate Orders and 7 Days of User Session Data Preceding Each Order
- Our goal is to create a weight for each session per order, and to assign the least weight to the middle session(s).
- To simplify demonstrating the model, I created a toy data set with the following fields:
    - `OrderID`
    - `OrderDT` - Order Date and Time
    - `SessionID` - A unique identifyer for a time-bound bundle of user clicks, associated with a single `ChannelID`
    - `SessionDT` - Session Date and Time
    - `ChannelID`
- We are looking at these two orders and a 7-day window of User Sessions leading up to each.
- There are an ODD number (11 sessions) associated with order #123, and an EVEN number (10 sessions) associated with #456

In [149]:
# Import Necessary Modules
import pandas as pd
import sqlite3
import csv

#### Create a file `attrib_model.db` in the folder you're working in

In [168]:
# To get data from a csv into a .db file:
conn = sqlite3.connect('attrib_model.sqlite') # NOTE: Make sure you first create a blank "attrib_model.sqlite" file in the folder you're working in
cursor = conn.cursor()

In [151]:
create_table_query = """
CREATE TABLE clicks AS 
(
ordernumber text, 
orderdatetime text, 
sessionid text, 
sessiondatetime text, 
channel text
)
"""


In [152]:
# Create Pandas dataframe from csv data:
clicks = pd.read_csv('attrib_model_toy_data.csv')
clicks.to_sql('clicks', conn, if_exists='replace', index = False)

In [153]:
pd.read_sql('''SELECT * FROM clicks''', conn)

,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel
0,123,2023-07-23 17:00:00,ihg789,2023-07-12 17:00:00,OrganicSearch
1,123,2023-07-23 17:00:00,lkj123,2023-07-13 18:00:00,OrganicSearch
2,123,2023-07-23 17:00:00,onm654,2023-07-14 19:00:00,OrganicSearch
3,123,2023-07-23 17:00:00,rqp789,2023-07-15 20:00:00,Email
4,123,2023-07-23 17:00:00,uts123,2023-07-16 21:00:00,Email
5,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch
6,123,2023-07-23 17:00:00,zzy789,2023-07-18 23:00:00,Direct
7,123,2023-07-23 17:00:00,cba321,2023-07-19 00:00:00,Social
8,123,2023-07-23 17:00:00,fed654,2023-07-20 01:00:00,OrganicSearch
9,123,2023-07-23 17:00:00,ghi987,2023-07-21 02:00:00,Email


#### Bifurcase Sessions per Order Using NTILE
- I bifurcate the sessions, partitioned by `OrderID` (ordered by `SessionStartTime`) into two groups using the `NTILE` function
- For each order, the first half of events are `NTILE` = 1, second half is `NTILE` = 2
    - In the case of an order having an odd-number of sessions associated with it, there will be slightly more sessions where `NTILE` = 1 versus `NTILE` = 2

In [154]:
# ntile_clicks query:
ntile_query = '''
DROP TABLE IF EXISTS ntile_clicks;
CREATE TABLE ntile_clicks AS SELECT
OrderNumber,
OrderDateTime,
SessionID,
SessionDateTime,
Channel,
ROW_NUMBER() OVER (PARTITION BY OrderNumber
    ORDER BY SessionDateTime) AS RowNumber,
    NTILE(2) OVER (PARTITION BY OrderNumber ORDER BY SessionDateTime) AS Ntile
FROM
clicks;
'''

In [155]:
# ntile_clicks execute and read table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(ntile_query)

pd.read_sql('''
SELECT * FROM ntile_clicks
''', conn)

,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile
0,123,2023-07-23 17:00:00,ihg789,2023-07-12 17:00:00,OrganicSearch,1,1
1,123,2023-07-23 17:00:00,lkj123,2023-07-13 18:00:00,OrganicSearch,2,1
2,123,2023-07-23 17:00:00,onm654,2023-07-14 19:00:00,OrganicSearch,3,1
3,123,2023-07-23 17:00:00,rqp789,2023-07-15 20:00:00,Email,4,1
4,123,2023-07-23 17:00:00,uts123,2023-07-16 21:00:00,Email,5,1
5,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch,6,1
6,123,2023-07-23 17:00:00,zzy789,2023-07-18 23:00:00,Direct,7,2
7,123,2023-07-23 17:00:00,cba321,2023-07-19 00:00:00,Social,8,2
8,123,2023-07-23 17:00:00,fed654,2023-07-20 01:00:00,OrganicSearch,9,2
9,123,2023-07-23 17:00:00,ghi987,2023-07-21 02:00:00,Email,10,2


- Next I count distinct `SessionID` per Order using `LAST_VALUE` and partitioning by `OrderNumber`


In [156]:
# last_val_clicks query:
last_val_query = '''
DROP TABLE IF EXISTS last_val_clicks;
CREATE TABLE last_val_clicks AS SELECT
OrderNumber,
OrderDateTime,
SessionID,
SessionDateTime,
Channel,
RowNumber,
Ntile,           
LAST_VALUE ( RowNumber)
    OVER ( PARTITION BY OrderNumber
    ORDER BY SessionDateTime ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS EventCount
FROM ntile_clicks;
    '''

In [157]:
# last_val_clicks execute and read table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(last_val_query)

pd.read_sql('''
SELECT * FROM last_val_clicks
''', conn)

,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile,EventCount
0,123,2023-07-23 17:00:00,ihg789,2023-07-12 17:00:00,OrganicSearch,1,1,11
1,123,2023-07-23 17:00:00,lkj123,2023-07-13 18:00:00,OrganicSearch,2,1,11
2,123,2023-07-23 17:00:00,onm654,2023-07-14 19:00:00,OrganicSearch,3,1,11
3,123,2023-07-23 17:00:00,rqp789,2023-07-15 20:00:00,Email,4,1,11
4,123,2023-07-23 17:00:00,uts123,2023-07-16 21:00:00,Email,5,1,11
5,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch,6,1,11
6,123,2023-07-23 17:00:00,zzy789,2023-07-18 23:00:00,Direct,7,2,11
7,123,2023-07-23 17:00:00,cba321,2023-07-19 00:00:00,Social,8,2,11
8,123,2023-07-23 17:00:00,fed654,2023-07-20 01:00:00,OrganicSearch,9,2,11
9,123,2023-07-23 17:00:00,ghi987,2023-07-21 02:00:00,Email,10,2,11


#### Split orders into odd and even group:
- From there I split the orders data into those with an odd and even number of events per order
    - For orders having ODD number of events: I re-assign the middle event (ordered by `SessionStartTime`) in the field `NTILE` to 3.
        - Now `NTILE` = 1 for the first half of the events, 3 for the middle event, and 2 for the second half of the events:
        - `CASE WHEN RowNumber = EventCount / 2 + 1 THEN 3 ELSE Ntile END AS Ntile`

#### Odd Group:

- Continuing with the odd group:
    - For each order number, assign a row number (Named it `NtileRow`) starting with the middle event (`NTILE` = 3) and ascending in both directions to the first and last events, respectively
    - For example, for an order that has eleven User Event Sessions (`EventSessionKey`), it would have these values for `NtileRow`

In [158]:
# Create odd_sessions query:
odd_sessions_query = '''
DROP TABLE IF EXISTS odd_sessions;
CREATE TABLE odd_sessions AS SELECT
OrderNumber, 
OrderDateTime, 
SessionID, 
SessionDateTime, 
Channel, 
RowNumber, 
Ntile, 
EventCount
FROM last_val_clicks
WHERE
(EventCount % 2) != 0;
  '''


In [159]:
# last_val_clicks execute and read table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(odd_sessions_query)

pd.read_sql('''
SELECT * FROM odd_sessions
''', conn)

,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile,EventCount
0,123,2023-07-23 17:00:00,ihg789,2023-07-12 17:00:00,OrganicSearch,1,1,11
1,123,2023-07-23 17:00:00,lkj123,2023-07-13 18:00:00,OrganicSearch,2,1,11
2,123,2023-07-23 17:00:00,onm654,2023-07-14 19:00:00,OrganicSearch,3,1,11
3,123,2023-07-23 17:00:00,rqp789,2023-07-15 20:00:00,Email,4,1,11
4,123,2023-07-23 17:00:00,uts123,2023-07-16 21:00:00,Email,5,1,11
5,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch,6,1,11
6,123,2023-07-23 17:00:00,zzy789,2023-07-18 23:00:00,Direct,7,2,11
7,123,2023-07-23 17:00:00,cba321,2023-07-19 00:00:00,Social,8,2,11
8,123,2023-07-23 17:00:00,fed654,2023-07-20 01:00:00,OrganicSearch,9,2,11
9,123,2023-07-23 17:00:00,ghi987,2023-07-21 02:00:00,Email,10,2,11


In [160]:
 # Re-assign Ntile so that middle number = 3:
odd_sessions_update_middle_query= '''
DROP TABLE IF EXISTS odd_sessions_update_middle;
CREATE TABLE odd_sessions_update_middle AS
SELECT
OrderNumber, 
OrderDateTime, 
SessionID, 
SessionDateTime, 
Channel, 
RowNumber, 
CASE WHEN RowNumber = EventCount / 2 + 1 THEN 3 ELSE Ntile END AS Ntile,
EventCount
FROM odd_sessions
'''

#  Read and Execute Table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(odd_sessions_update_middle_query)

pd.read_sql('''
SELECT * FROM odd_sessions_update_middle 
''', conn)

,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile,EventCount
0,123,2023-07-23 17:00:00,ihg789,2023-07-12 17:00:00,OrganicSearch,1,1,11
1,123,2023-07-23 17:00:00,lkj123,2023-07-13 18:00:00,OrganicSearch,2,1,11
2,123,2023-07-23 17:00:00,onm654,2023-07-14 19:00:00,OrganicSearch,3,1,11
3,123,2023-07-23 17:00:00,rqp789,2023-07-15 20:00:00,Email,4,1,11
4,123,2023-07-23 17:00:00,uts123,2023-07-16 21:00:00,Email,5,1,11
5,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch,6,3,11
6,123,2023-07-23 17:00:00,zzy789,2023-07-18 23:00:00,Direct,7,2,11
7,123,2023-07-23 17:00:00,cba321,2023-07-19 00:00:00,Social,8,2,11
8,123,2023-07-23 17:00:00,fed654,2023-07-20 01:00:00,OrganicSearch,9,2,11
9,123,2023-07-23 17:00:00,ghi987,2023-07-21 02:00:00,Email,10,2,11


In [161]:
# Assign the first half Ntile Row Number:
first_half_odd_query= '''
DROP TABLE IF EXISTS first_half_odd;
CREATE TABLE first_half_odd AS SELECT
OrderNumber, 
OrderDateTime, 
SessionID, 
SessionDateTime, 
Channel, 
RowNumber, 
Ntile,
  CASE WHEN Ntile = 1 OR Ntile = 3 THEN ROW_NUMBER () OVER ( PARTITION BY ORDERNUMBER ORDER BY SessionDateTime DESC) END AS NtileRow,
  EventCount
FROM
odd_sessions_update_middle
WHERE
  Ntile != 2;
'''

#  Read and Execute Table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(first_half_odd_query)

pd.read_sql('''
SELECT * FROM first_half_odd 
''', conn)

,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile,NtileRow,EventCount
0,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch,6,3,1,11
1,123,2023-07-23 17:00:00,uts123,2023-07-16 21:00:00,Email,5,1,2,11
2,123,2023-07-23 17:00:00,rqp789,2023-07-15 20:00:00,Email,4,1,3,11
3,123,2023-07-23 17:00:00,onm654,2023-07-14 19:00:00,OrganicSearch,3,1,4,11
4,123,2023-07-23 17:00:00,lkj123,2023-07-13 18:00:00,OrganicSearch,2,1,5,11
5,123,2023-07-23 17:00:00,ihg789,2023-07-12 17:00:00,OrganicSearch,1,1,6,11


In [162]:
# Assign the second half Ntile Row Number:
second_half_odd_query= '''
DROP TABLE IF EXISTS second_half_odd;
CREATE TABLE second_half_odd AS SELECT
OrderNumber, 
OrderDateTime, 
SessionID, 
SessionDateTime, 
Channel, 
RowNumber, 
Ntile,
CASE WHEN Ntile = 2 OR Ntile = 3 THEN ROW_NUMBER () OVER ( PARTITION BY OrderNumber ORDER BY SessionDateTime) END AS NtileRow,
EventCount
FROM
odd_sessions_update_middle
WHERE
  Ntile != 1;
'''

#  Read and Execute Table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(second_half_odd_query)

pd.read_sql('''
SELECT * FROM second_half_odd 
''', conn)

,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile,NtileRow,EventCount
0,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch,6,3,1,11
1,123,2023-07-23 17:00:00,zzy789,2023-07-18 23:00:00,Direct,7,2,2,11
2,123,2023-07-23 17:00:00,cba321,2023-07-19 00:00:00,Social,8,2,3,11
3,123,2023-07-23 17:00:00,fed654,2023-07-20 01:00:00,OrganicSearch,9,2,4,11
4,123,2023-07-23 17:00:00,ghi987,2023-07-21 02:00:00,Email,10,2,5,11
5,123,2023-07-23 17:00:00,fed456,2023-07-22 03:00:00,PaidSearch,11,2,6,11


In [163]:
# Union first and second halves:
union_odd_halves_query= '''
DROP TABLE IF EXISTS union_odd_halves;
CREATE TABLE union_odd_halves AS 
SELECT * FROM first_half_odd UNION SELECT * from second_half_odd ORDER BY RowNumber;
'''
#  Read and Execute Table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(union_odd_halves_query)

pd.read_sql('''
SELECT * FROM union_odd_halves 
''', conn)

,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile,NtileRow,EventCount
0,123,2023-07-23 17:00:00,ihg789,2023-07-12 17:00:00,OrganicSearch,1,1,6,11
1,123,2023-07-23 17:00:00,lkj123,2023-07-13 18:00:00,OrganicSearch,2,1,5,11
2,123,2023-07-23 17:00:00,onm654,2023-07-14 19:00:00,OrganicSearch,3,1,4,11
3,123,2023-07-23 17:00:00,rqp789,2023-07-15 20:00:00,Email,4,1,3,11
4,123,2023-07-23 17:00:00,uts123,2023-07-16 21:00:00,Email,5,1,2,11
5,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch,6,3,1,11
6,123,2023-07-23 17:00:00,zzy789,2023-07-18 23:00:00,Direct,7,2,2,11
7,123,2023-07-23 17:00:00,cba321,2023-07-19 00:00:00,Social,8,2,3,11
8,123,2023-07-23 17:00:00,fed654,2023-07-20 01:00:00,OrganicSearch,9,2,4,11
9,123,2023-07-23 17:00:00,ghi987,2023-07-21 02:00:00,Email,10,2,5,11


#### Even Group:
- Even Group:
    - Assigning an `NtileRow` value is much more straightforward
    - We assign the group where `Ntile` = 1 an `NtileRow` value by ordering `SessionStartTime` descending, and the Ntile = 2 group ordering `SessionStartTime` ascending

In [164]:
# Create even_sessions query:
even_sessions_query = '''
DROP TABLE IF EXISTS even_sessions;
CREATE TABLE even_sessions AS SELECT
OrderNumber, 
OrderDateTime, 
SessionID, 
SessionDateTime, 
Channel, 
RowNumber, 
Ntile, 
  CASE WHEN Ntile = 1 THEN ROW_NUMBER() OVER (PARTITION BY OrderNumber, Ntile ORDER BY SessionDateTime DESC)
       ELSE ROW_NUMBER() OVER (PARTITION BY OrderNumber, Ntile ORDER BY SessionDateTime) END AS NtileRow,
  EventCount
FROM last_val_clicks
WHERE
(EventCount % 2) = 0
ORDER BY RowNumber;
  '''
#  Read and Execute Table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(even_sessions_query)

pd.read_sql('''
SELECT * FROM even_sessions
''', conn)

,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile,NtileRow,EventCount
0,456,2023-07-24 16:00:00,abc123,2023-07-13 17:00:00,OrganicSearch,1,1,5,10
1,456,2023-07-24 16:00:00,def456,2023-07-14 18:00:00,Social,2,1,4,10
2,456,2023-07-24 16:00:00,ghi789,2023-07-16 19:00:00,OrganicSearch,3,1,3,10
3,456,2023-07-24 16:00:00,jkl123,2023-07-17 20:00:00,PaidSearch,4,1,2,10
4,456,2023-07-24 16:00:00,mno456,2023-07-18 21:00:00,Direct,5,1,1,10
5,456,2023-07-24 16:00:00,pqr789,2023-07-19 22:00:00,Direct,6,2,1,10
6,456,2023-07-24 16:00:00,stu123,2023-07-20 23:00:00,Social,7,2,2,10
7,456,2023-07-24 16:00:00,vwx456,2023-07-21 00:00:00,PaidSearch,8,2,3,10
8,456,2023-07-24 16:00:00,yzz789,2023-07-22 01:00:00,PaidSearch,9,2,4,10
9,456,2023-07-24 16:00:00,cba123,2023-07-23 02:00:00,Email,10,2,5,10


In [165]:
# Union even and odd tables:
union_even_odd_query = '''
DROP TABLE IF EXISTS union_even_odd;
CREATE TABLE union_even_odd AS 
SELECT * FROM even_sessions UNION SELECT * from union_odd_halves ORDER BY OrderNumber, RowNumber;
'''
#  Read and Execute Table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(union_even_odd_query)

pd.read_sql('''
SELECT * FROM  union_even_odd 
''', conn)

,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile,NtileRow,EventCount
0,123,2023-07-23 17:00:00,ihg789,2023-07-12 17:00:00,OrganicSearch,1,1,6,11
1,123,2023-07-23 17:00:00,lkj123,2023-07-13 18:00:00,OrganicSearch,2,1,5,11
2,123,2023-07-23 17:00:00,onm654,2023-07-14 19:00:00,OrganicSearch,3,1,4,11
3,123,2023-07-23 17:00:00,rqp789,2023-07-15 20:00:00,Email,4,1,3,11
4,123,2023-07-23 17:00:00,uts123,2023-07-16 21:00:00,Email,5,1,2,11
5,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch,6,3,1,11
6,123,2023-07-23 17:00:00,zzy789,2023-07-18 23:00:00,Direct,7,2,2,11
7,123,2023-07-23 17:00:00,cba321,2023-07-19 00:00:00,Social,8,2,3,11
8,123,2023-07-23 17:00:00,fed654,2023-07-20 01:00:00,OrganicSearch,9,2,4,11
9,123,2023-07-23 17:00:00,ghi987,2023-07-21 02:00:00,Email,10,2,5,11


#### The math:
- I now have my field `NtileRow` that I can use to create a weight where the middle event is weighted the least, reverse decaying to the first and last events
- First I square `NtileRow` and sum the squares partitioned by `OrderNumber`:

In [166]:
# Create square_ntile_row query:: 
square_ntile_row_query = '''
DROP TABLE IF EXISTS square_ntile_row;
CREATE TABLE square_ntile_row AS
SELECT
OrderNumber,
OrderDateTime,
SessionID,
SessionDateTime,
Channel,
RowNumber,
Ntile,
  NtileRow,
  (NtileRow*NtileRow) AS NtileRowSq,
  SUM(NtileRow*NtileRow) OVER (PARTITION BY OrderNumber ORDER BY SessionDateTime ROWS UNBOUNDED PRECEDING) AS SumNtileRowSq,
EventCount
FROM union_even_odd
'''

#  Read and Execute Table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(square_ntile_row_query)

pd.read_sql('''
SELECT * FROM square_ntile_row 
''', conn)



,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile,NtileRow,NtileRowSq,SumNtileRowSq,EventCount
0,123,2023-07-23 17:00:00,ihg789,2023-07-12 17:00:00,OrganicSearch,1,1,6,36,36,11
1,123,2023-07-23 17:00:00,lkj123,2023-07-13 18:00:00,OrganicSearch,2,1,5,25,61,11
2,123,2023-07-23 17:00:00,onm654,2023-07-14 19:00:00,OrganicSearch,3,1,4,16,77,11
3,123,2023-07-23 17:00:00,rqp789,2023-07-15 20:00:00,Email,4,1,3,9,86,11
4,123,2023-07-23 17:00:00,uts123,2023-07-16 21:00:00,Email,5,1,2,4,90,11
5,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch,6,3,1,1,91,11
6,123,2023-07-23 17:00:00,zzy789,2023-07-18 23:00:00,Direct,7,2,2,4,95,11
7,123,2023-07-23 17:00:00,cba321,2023-07-19 00:00:00,Social,8,2,3,9,104,11
8,123,2023-07-23 17:00:00,fed654,2023-07-20 01:00:00,OrganicSearch,9,2,4,16,120,11
9,123,2023-07-23 17:00:00,ghi987,2023-07-21 02:00:00,Email,10,2,5,25,145,11


#### Assign weight to each event:
- Finally, we get our "parabola shaped" multi-click attribution model `weight` value
    - The formula is `NtileRowSq * (1.0 / Sum(NtileRowSq))` (The sum of `NtileRowSq` per order is the last value partitioned by `OrderNumber` in the `SumNtileRowSq` field:
    - This `weight` field was then available in our "Click-Stream" marketing model to evaluate the estimated impact of our marketing campaigns on purchases.

In [167]:
# Create parabolic_multi_touch query:: 
parabolic_multi_touch_query = '''
DROP TABLE IF EXISTS parabolic_multi_touch;
CREATE TEMP TABLE parabolic_multi_touch AS
SELECT
OrderNumber,
OrderDateTime,
SessionID,
SessionDateTime,
Channel,
RowNumber,
Ntile,
NtileRow,
 NtileRowSq,
 SumNtileRowSq,
 NtileRowSq * (1.0 / LAST_VALUE(SumNtileRowSq) OVER 
(PARTITION BY OrderNumber ORDER BY SessionDateTime ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)) AS Weight,
EventCount
FROM
 square_ntile_row 
'''

#  Read and Execute Table:
with sqlite3.connect('attrib_model.sqlite') as conn:
     conn.executescript(parabolic_multi_touch_query)

pd.read_sql('''
SELECT * FROM parabolic_multi_touch 
''', conn)


,OrderNumber,OrderDateTime,SessionID,SessionDateTime,Channel,RowNumber,Ntile,NtileRow,NtileRowSq,SumNtileRowSq,Weight,EventCount
0,123,2023-07-23 17:00:00,ihg789,2023-07-12 17:00:00,OrganicSearch,1,1,6,36,36,0.198895,11
1,123,2023-07-23 17:00:00,lkj123,2023-07-13 18:00:00,OrganicSearch,2,1,5,25,61,0.138122,11
2,123,2023-07-23 17:00:00,onm654,2023-07-14 19:00:00,OrganicSearch,3,1,4,16,77,0.088398,11
3,123,2023-07-23 17:00:00,rqp789,2023-07-15 20:00:00,Email,4,1,3,9,86,0.049724,11
4,123,2023-07-23 17:00:00,uts123,2023-07-16 21:00:00,Email,5,1,2,4,90,0.022099,11
5,123,2023-07-23 17:00:00,xwv456,2023-07-17 22:00:00,PaidSearch,6,3,1,1,91,0.005525,11
6,123,2023-07-23 17:00:00,zzy789,2023-07-18 23:00:00,Direct,7,2,2,4,95,0.022099,11
7,123,2023-07-23 17:00:00,cba321,2023-07-19 00:00:00,Social,8,2,3,9,104,0.049724,11
8,123,2023-07-23 17:00:00,fed654,2023-07-20 01:00:00,OrganicSearch,9,2,4,16,120,0.088398,11
9,123,2023-07-23 17:00:00,ghi987,2023-07-21 02:00:00,Email,10,2,5,25,145,0.138122,11


- As you can see above, the middle User Event Session, where `EventID` = 6, has the least weight, and the weight symmetrically and exponentially increases.
    - Where there is an EVEN number of User Event Sessions per Order, we have two middle events with the same weight.